<a href="https://colab.research.google.com/github/tmandingwa/tmandingwa/blob/main/Inclass_Knapsack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 A Knapsack problem

This notebook demonstrates how to formulate and solve a capital budgeting optimization problem using Python and Pyomo. The goal is to select the combination of projects that maximizes net present value (NPV) under a budget constraint based on an input Excel dataset (knapsack problem - what goes into the bag).

In [ ]:
# The usual installation of packages.
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
import pandas as pd
import pyomo.environ as pyo


Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 125080 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

# 1.1 Playground to develop an optimization problem for the knapsack

In [ ]:
# Upload file and read data

# Create a model

# Set up the variables

# Set up objective

# Set up constraints

# Solve

# Output results

# 2. Solution to the Knapsack problem.

In [ ]:
# Upload a file in your local machine to Colab
from google.colab import files
uploaded = files.upload()

# Read data from xlsx file
df = pd.read_excel("capital_budgeting.xlsx", index_col=0, usecols="A:F", nrows=3)
df

Saving capital_budgeting.xlsx to capital_budgeting.xlsx


,P1,P2,P3,P4,P5
NPV,10,17,16,8,14
Expenditure,48,96,80,32,64


## 2.1 Model Definition

We create a Pyomo concrete model, declare the set of projects, and define binary decision variables indicating whether each project is selected.

In [ ]:
model = pyo.ConcreteModel(name='Capital Budgeting')

# Projects set from DataFrame columns
model.projects = pyo.Set(initialize=df.columns)

# Binary decision variable: 1 if project is selected, 0 otherwise
model.x = pyo.Var(model.projects, domain=pyo.Binary)

# Budget limit for the selection
model.budget_limit = 160

## 2.2 Objective Function

Define an objective to maximize the total NPV of selected projects.

In [ ]:
def total_npv(model):
    return sum(df.at['NPV', j] * model.x[j] for j in model.projects)

model.total_npv = pyo.Objective(rule=total_npv, sense=pyo.maximize)

## 2.3 Budget Constraint

Enforce that total expenditure of selected projects does not exceed the budget limit.

In [ ]:
# Do this yourself. HINT: It looks structurally similar to the objective function.
# Also remember that you can hardcode.

# 2.3 Budget Constraint
# ---------------------
# This ensures total expenditure of selected projects does not exceed the budget limit.

def budget_rule(model):
    # Sum of expenditure for selected projects must be ≤ 160
    return sum(df.at['Expenditure', j] * model.x[j] for j in model.projects) <= model.budget_limit

# Add the constraint to the model
model.budget_constr = pyo.Constraint(rule=budget_rule)



## 2.4 Solving the Model

Use the GLPK solver to solve the optimization problem.

In [ ]:
solver = pyo.SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 34.0, 'Upper bound': 34.0, 'Number of objectives': 1, 'Number of constraints': 1, 'Number of variables': 5, 'Number of nonzeros': 5, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '5', 'Number of created subproblems': '5'}}, 'Error rc': 0, 'Time': 0.004104137420654297}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

## 2.5 Results Extraction and Export

Extract the decision variable values, append them to the DataFrame under a 'Result' row, export to Excel, and display the model summary.

In [ ]:
df_assign = df.copy()
for j in model.projects:
    df_assign.at['Result', j] = model.x[j].value

# Save the results
df_assign.to_excel('budget_output.xlsx', index=True)

# Display the model summary
df_assign

,P1,P2,P3,P4,P5
NPV,10.0,17.0,16.0,8.0,14.0
Expenditure,48.0,96.0,80.0,32.0,64.0
Result,1.0,0.0,1.0,1.0,0.0


In [ ]:
df_assign = df.copy()
for j in model.projects:
    df_assign.at['Result', j] = model.x[j].value

# Save the results
df_assign.to_excel('budget_output.xlsx', index=True)

# Display the model summary
df_assign

,P1,P2,P3,P4,P5
NPV,10.0,17.0,16.0,8.0,14.0
Expenditure,48.0,96.0,80.0,32.0,64.0
Result,1.0,0.0,1.0,1.0,0.0
